## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [27]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
import cv2
from sklearn.preprocessing import OneHotEncoder


In [28]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print(x_train.shape) #(50000, 32, 32, 3)
print(x_train[0].shape) #(50000, 32, 32, 3)
# cv2.imshow('R', x_train[100])
## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        print('mean:',mean)  
        print('std:',std)  
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

50000 train samples
10000 test samples
(50000, 32, 32, 3)
(32, 32, 3)
mean: 120.70756512369792
std: 64.1500758911213


In [29]:
y_train

array([[6],
       [9],
       [9],
       ...,
       [9],
       [1],
       [1]], dtype=uint8)

In [30]:
y_test

array([[3],
       [8],
       [8],
       ...,
       [5],
       [1],
       [7]])

In [31]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder(categories='auto')
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()


In [32]:
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [33]:
y_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

In [ ]:

classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32, kernel_size=(3, 3), padding='same',input_shape=x_train.shape[1:],activation='relu'))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())

##'''自己決定MaxPooling2D放在哪裡'''
#classifier.add(MaxPooling2D(pool_size=(2,2)))

##卷積組合
classifier.add(Convolution2D(32, kernel_size=(3, 3), padding='same',activation='relu'))
classifier.add(BatchNormalization())

##flatten
classifier.add(Flatten())

##FC
classifier.add(Dense(100)) #output_dim=100,activation=relu

##輸出
classifier.add(Dense(output_dim=10,activation='softmax'))

##超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=100)

C:\Users\WilsonChen\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`


Epoch 1/100
50000/50000 [==============================] - 195s 4ms/step - loss: 3.9624 - accuracy: 0.4952
Epoch 2/100
50000/50000 [==============================] - 195s 4ms/step - loss: 0.8525 - accuracy: 0.7046
Epoch 3/100
50000/50000 [==============================] - 186s 4ms/step - loss: 0.6990 - accuracy: 0.7555
Epoch 4/100
50000/50000 [==============================] - 181s 4ms/step - loss: 0.5738 - accuracy: 0.8001
Epoch 5/100
50000/50000 [==============================] - 199s 4ms/step - loss: 0.4728 - accuracy: 0.8334
Epoch 6/100
50000/50000 [==============================] - 209s 4ms/step - loss: 0.3865 - accuracy: 0.8630
Epoch 7/100
50000/50000 [==============================] - 207s 4ms/step - loss: 0.3315 - accuracy: 0.8824
Epoch 8/100
50000/50000 [==============================] - 200s 4ms/step - loss: 0.2686 - accuracy: 0.9050
Epoch 9/100
50000/50000 [==============================] - 193s 4ms/step - loss: 0.2453 - accuracy: 0.9127
Epoch 10/100
50000/50000 [===========

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [ ]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)